Cell 1 — Header, imports, paths, config

In [1]:
# 02_model1_risk_classification.ipynb
# Goal: Train a calibrated, robust classifier that predicts risk (LOW/MEDIUM/HIGH)
# using deployable signals: Temperature, pH, Turbidity_proxy + Virtual DO/NH3.

from pathlib import Path
import os, json, math, warnings, yaml
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import StratifiedKFold, train_test_split, cross_val_predict
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import (classification_report, confusion_matrix, f1_score,
                             average_precision_score, brier_score_loss)
from sklearn.calibration import CalibratedClassifierCV
from sklearn.utils.validation import check_is_fitted

# Models
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, GradientBoostingClassifier

import joblib

# ---------- Auto-detect POSEIDON root ----------
def find_project_root(project_name="POSEIDON"):
    cwd = Path.cwd().resolve()
     # check current dir and parents for a folder named "Poseidon" (case-insensitive)
    project_name_l = project_name.lower()
    for p in [cwd] + list(cwd.parents):
        if p.name.lower() == project_name_l:
            return p
    # fallback: if running from 'notebooks' use its parent
    if cwd.name.lower() == "notebooks" and cwd.parent.exists():
        return cwd.parent
    raise FileNotFoundError(f"Could not locate project root '{project_name}'. Starting cwd: {cwd}")

    '''
    if cwd.name == project_name:
        return cwd
    if (cwd / project_name).exists():
        return cwd / project_name
    for parent in [cwd] + list(cwd.parents):
        cand = parent / project_name
        if cand.exists():
            return cand
    raise FileNotFoundError(f"Could not locate project root '{project_name}'. Starting cwd: {cwd}")
    '''

ROOT = find_project_root("POSEIDON")

# ---------- Standard project folders ----------
DATA      = ROOT / "data"
INTERIM   = DATA / "interim"
ART       = ROOT / "artifacts"
MODEL_REG = ART  / "model_registry"
CVR       = ART  / "cv_reports"
HOLD      = ART  / "holdout_reports"
EXPL      = ART  / "explainability"
CONFIGS   = ROOT / "configs"

for d in [MODEL_REG, CVR, HOLD, EXPL]:
    d.mkdir(parents=True, exist_ok=True)

# ---------- Load thresholds config ----------
cfg_path = CONFIGS / "thresholds.yml"
if not cfg_path.exists():
    raise FileNotFoundError(
        f"Missing {cfg_path}. Create it (we provided a template earlier) then rerun."
    )

with open(cfg_path, "r") as f:
    CFG_THRESH = yaml.safe_load(f)

MODEL_VERSION = "model1_risk_classifier_v1"
RANDOM_SEED  = 42

print("ROOT:", ROOT)
print("Using thresholds file:", cfg_path)
print("DATA:", DATA)
print("ART:", ART)


ROOT: C:\Users\PC\Documents\Machine_Learning\Capstone_Project\Poseidon
Using thresholds file: C:\Users\PC\Documents\Machine_Learning\Capstone_Project\Poseidon\configs\thresholds.yml
DATA: C:\Users\PC\Documents\Machine_Learning\Capstone_Project\Poseidon\data
ART: C:\Users\PC\Documents\Machine_Learning\Capstone_Project\Poseidon\artifacts


Cell 2 — Load cleaned WQD and virtual sensor models

In [2]:
# Load WQD (clean) produced by 00_data_understanding_eda.ipynb
wqd_path = INTERIM / "wqd_clean.csv"
if not wqd_path.exists():
    raise FileNotFoundError(f"Missing {wqd_path}. Run 00_data_understanding_eda.ipynb first.")
wqd = pd.read_csv(wqd_path)

# Load virtual sensors produced by 01_soft_sensors_DO_NH3.ipynb
soft_dir    = ART / "soft_sensors"
vm_do_path  = soft_dir / "virtual_do.joblib"
vm_nh3_path = soft_dir / "virtual_nh3.joblib"

print("Looking for virtual sensors in:", soft_dir)
print("Files in soft_sensors:", [p.name for p in soft_dir.glob('*')])

if not (vm_do_path.exists() and vm_nh3_path.exists()):
    raise FileNotFoundError(
        "Virtual sensors not found.\n"
        f"Expected:\n  - {vm_do_path}\n  - {vm_nh3_path}\n"
        "→ Run 01_soft_sensors_DO_NH3.ipynb to train & save them."
    )

vm_do  = joblib.load(vm_do_path)   # {"model": mdl, "features": [...]}
vm_nh3 = joblib.load(vm_nh3_path)

print("WQD shape:", wqd.shape)
display(wqd.head(3))
print("VM DO features:", vm_do["features"])
print("VM NH3 features:", vm_nh3["features"])


Looking for virtual sensors in: C:\Users\PC\Documents\Machine_Learning\Capstone_Project\Poseidon\artifacts\soft_sensors
Files in soft_sensors: ['virtual_do.joblib', 'virtual_models_metrics.json', 'virtual_nh3.joblib']
WQD shape: (4300, 7)


,temperature,turbidity_cm,do,pH,ammonia,water_quality,turbidity_proxy
0,67.448725,10.127148,0.208153,4.751657,0.286054,2,0.098744
1,64.626666,94.015595,11.434463,3.085154,0.096040,2,0.010637
2,65.121842,90.653462,12.430865,9.648515,0.974501,2,0.011031


VM DO features: ['temperature', 'pH', 'turbidity_proxy']
VM NH3 features: ['temperature', 'pH', 'turbidity_proxy']


Cell 3 — Build deployable features (Temp, pH, Turbidity_proxy + Virtual DO/NH3)

In [3]:
# Deploy-time base inputs (your actual field sensors)
INPUTS_DEPLOY = ["temperature", "pH", "turbidity_proxy"]

# Base matrix
X_base = wqd[INPUTS_DEPLOY].copy()

# Predict virtual DO/NH3 strictly from deployable inputs
Xv = X_base[vm_do["features"]].values  # same for DO and NH3 models
feat = X_base.copy()
feat["predicted_do"]  = vm_do["model"].predict(Xv)
feat["predicted_nh3"] = vm_nh3["model"].predict(Xv)

print("Feature snapshot:")
display(feat.head(5))


Feature snapshot:


,temperature,pH,turbidity_proxy,predicted_do,predicted_nh3
0,67.448725,4.751657,0.098744,5.271772,0.286054
1,64.626666,3.085154,0.010637,4.957571,0.432427
2,65.121842,9.648515,0.011031,10.173653,0.796088
3,1.640334,4.819988,15.072963,7.010061,0.475290
4,64.863434,10.244034,0.471882,5.607760,0.444645


Risk labels from REAL DO/NH₃ using config policy (no leakage)

In [4]:
# Risk labeling from REAL DO/NH3 (no circularity). Config-driven.

def label_from_config(row, cfg, q_high_turb):
    score = 0
    LP = cfg["label_policy"]["scores"]

    # DO (lower is worse)
    do_val   = row["do"]
    do_crit  = cfg["do"]["primary"]["critical"]
    do_low   = cfg["do"]["primary"]["low"]
    if do_val < do_crit:
        score += LP["do"]["critical"]
    elif do_val < do_low:
        score += LP["do"]["low"]

    # NH3 (higher is worse)
    nh3_val  = row["ammonia"]
    nh3_high = cfg["nh3"]["primary"]["high"]
    nh3_elev = cfg["nh3"]["primary"]["elevated"]
    if nh3_val > nh3_high:
        score += LP["nh3"]["high"]
    elif nh3_val > nh3_elev:
        score += LP["nh3"]["elevated"]

    # pH comfort band
    if (row["pH"] < cfg["pH"]["lo"]) or (row["pH"] > cfg["pH"]["hi"]):
        score += LP["pH_out_of_range"]

    # turbidity proxy (relative high)
    if row["turbidity_proxy"] > q_high_turb:
        score += LP["turbidity_high"]

    # temperature stress
    if row["temperature"] > cfg["temperature"]["hi"]:
        score += LP["temperature_high"]

    # score -> label mapping (descending by min_score)
    for rule in cfg["label_policy"]["scores_to_labels"]:
        if score >= rule["min_score"]:
            return rule["label"]
    return "LOW"

# Compute relative turbidity threshold from WQD
q_mark = wqd["turbidity_proxy"].quantile(CFG_THRESH["turbidity_proxy"]["quantile_high"])
labels = wqd.apply(lambda r: label_from_config(r, CFG_THRESH, q_mark), axis=1)

print("Label balance (proportion):")
display(labels.value_counts(normalize=True).rename("proportion").to_frame())


Label balance (proportion):


,proportion
MEDIUM,0.554884
LOW,0.401163
HIGH,0.043953


Cell 5 — Final training frame + stratified split

In [5]:
# Final features for classification
FEAT_COLS = ["temperature", "pH", "turbidity_proxy", "predicted_do", "predicted_nh3"]
X = feat[FEAT_COLS].copy()
y = labels.values

# Stratified split to preserve class ratios
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=RANDOM_SEED, stratify=y
)

print("Train:", X_train.shape, " Test:", X_test.shape)
print("Train label proportions:")
display(pd.Series(y_train).value_counts(normalize=True).rename("train_prop").to_frame())


Train: (3440, 5)  Test: (860, 5)
Train label proportions:


,train_prop
MEDIUM,0.554942
LOW,0.401163
HIGH,0.043895


Cell 6 — Metrics helpers (macro-AP, etc.)

In [6]:
classes = np.unique(y_train)
print("Detected classes:", classes)

def macro_ap_score(y_true, proba, class_list):
    """
    Macro-averaged one-vs-rest Average Precision.
    y_true: labels (array-like)
    proba : shape (n_samples, n_classes) aligned to class_list order
    """
    y_true_series = pd.Series(y_true)
    Y = pd.get_dummies(y_true_series).reindex(columns=class_list, fill_value=0).values
    return float(average_precision_score(Y, proba, average="macro"))


Detected classes: ['HIGH' 'LOW' 'MEDIUM']


Cell 7 — Candidates (10) + 5-fold CV leaderboard (class_weight='balanced')

In [7]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_SEED)
class_list = list(classes)

candidates = {
    "logreg_en": Pipeline([
        ("scaler", StandardScaler()),
        ("clf", LogisticRegression(
            max_iter=1000, multi_class="multinomial",
            class_weight="balanced", C=1.5, penalty="l2"
        ))
    ]),

    "linear_svc": Pipeline([
        ("scaler", StandardScaler()),
        ("clf", LinearSVC(class_weight="balanced"))
    ]),  # no predict_proba -> softmax decision_function

    "svm_rbf": Pipeline([
        ("scaler", StandardScaler()),
        ("clf", SVC(kernel="rbf", probability=True,
                    class_weight="balanced", C=2.0, gamma="scale"))
    ]),

    "knn": Pipeline([
        ("scaler", StandardScaler()),
        ("clf", KNeighborsClassifier(n_neighbors=11))
    ]),

    "gnb": GaussianNB(),

    "dtree": DecisionTreeClassifier(
        max_depth=8, class_weight="balanced", random_state=RANDOM_SEED
    ),

    "rf": RandomForestClassifier(
        n_estimators=400, max_depth=None, class_weight="balanced",
        random_state=RANDOM_SEED, n_jobs=-1
    ),

    "extra": ExtraTreesClassifier(
        n_estimators=500, class_weight="balanced",
        random_state=RANDOM_SEED, n_jobs=-1
    ),

    "gbrt": GradientBoostingClassifier(random_state=RANDOM_SEED),

    "rf_depth_limited": RandomForestClassifier(
        n_estimators=400, max_depth=16, class_weight="balanced",
        random_state=RANDOM_SEED, n_jobs=-1
    ),
}

leaderboard = []
for name, pipe in candidates.items():
    print(f"Evaluating {name} ...")

    # Probabilities for macro-AP
    if name == "linear_svc":
        dec = cross_val_predict(pipe, X_train, y_train, cv=skf, method="decision_function")
        if dec.ndim == 1:  # binary safety
            dec = np.vstack([-dec, dec]).T
        dec = dec - dec.max(axis=1, keepdims=True)
        proba = np.exp(dec)
        proba = proba / proba.sum(axis=1, keepdims=True)
    else:
        proba = cross_val_predict(pipe, X_train, y_train, cv=skf, method="predict_proba")

    # CV predictions (labels) for macro-F1
    y_pred = cross_val_predict(pipe, X_train, y_train, cv=skf, method="predict")

    ap = macro_ap_score(y_train, proba, class_list)
    f1 = f1_score(y_train, y_pred, average="macro")

    leaderboard.append({"model": name, "cv_macro_AP": ap, "cv_macro_F1": f1})

leader_df = pd.DataFrame(leaderboard).sort_values(["cv_macro_AP","cv_macro_F1"], ascending=False)
display(leader_df)

CVR.mkdir(parents=True, exist_ok=True)
leader_df.to_csv(CVR / "model1_cv_leaderboard.csv", index=False)


Evaluating logreg_en ...
Evaluating linear_svc ...
Evaluating svm_rbf ...
Evaluating knn ...
Evaluating gnb ...
Evaluating dtree ...
Evaluating rf ...
Evaluating extra ...
Evaluating gbrt ...
Evaluating rf_depth_limited ...


,model,cv_macro_AP,cv_macro_F1
8,gbrt,0.840487,0.781469
9,rf_depth_limited,0.833735,0.775760
6,rf,0.831555,0.776086
5,dtree,0.739064,0.710825
7,extra,0.738843,0.763419
3,knn,0.707375,0.661775
2,svm_rbf,0.671864,0.644628
1,linear_svc,0.638054,0.664228
0,logreg_en,0.631155,0.652501
4,gnb,0.625902,0.516893


Cell 8 — Pick best model → fit + calibrate (isotonic) on validation split

In [8]:
best_name = leader_df.iloc[0]["model"]
print("Top candidate:", best_name)

base_model = candidates[best_name]

# Split train into inner-train/val for calibration
X_tr, X_val, y_tr, y_val = train_test_split(
    X_train, y_train, test_size=0.2, random_state=RANDOM_SEED, stratify=y_train
)

# Fit base
base_model.fit(X_tr, y_tr)

# Calibrate probabilities (isotonic is accurate; sigmoid is a fallback for very small data)
calibrated = CalibratedClassifierCV(base_model, method="isotonic", cv="prefit")
calibrated.fit(X_val, y_val)

print("Calibrated classes:", calibrated.classes_)


Top candidate: gbrt
Calibrated classes: ['HIGH' 'LOW' 'MEDIUM']


Cell 9 — Threshold tuning (favor HIGH recall while keeping macro-F1 strong)

In [9]:
def find_thresholds_for_high(model, Xv, yv, classes):
    proba = model.predict_proba(Xv)
    idx = {c:i for i,c in enumerate(classes)}
    thresholds = {"HIGH": 0.5}

    if "HIGH" in idx:
        high_idx = idx["HIGH"]
        best = (0.5, 0.0)  # (thr, macro_f1)
        for t in np.linspace(0.30, 0.70, 21):
            y_hat = []
            for row in proba:
                if row[high_idx] >= t:
                    y_hat.append("HIGH")
                else:
                    y_hat.append(classes[np.argmax(row)])
            f1 = f1_score(yv, y_hat, average="macro")
            if f1 > best[1]:
                best = (float(t), float(f1))
        thresholds["HIGH"] = best[0]
    return thresholds

thresholds = find_thresholds_for_high(calibrated, X_val, y_val, calibrated.classes_)
print("Operating thresholds:", thresholds)

def predict_with_thresholds(model, X, thresholds, classes):
    proba = model.predict_proba(X)
    idx = {c:i for i,c in enumerate(classes)}
    y_hat = []
    for row in proba:
        if "HIGH" in idx and row[idx["HIGH"]] >= thresholds.get("HIGH", 0.5):
            y_hat.append("HIGH")
        else:
            y_hat.append(classes[np.argmax(row)])
    return np.array(y_hat), proba


Operating thresholds: {'HIGH': 0.3}


Cell 10 — Holdout evaluation (baseline vs thresholded)

In [10]:
def holdout_report(model, X_te, y_te, classes, title="[Baseline]"):
    y_pred = model.predict(X_te)
    proba  = model.predict_proba(X_te)

    cr = classification_report(y_te, y_pred, output_dict=True)
    cm = confusion_matrix(y_te, y_pred, labels=classes)
    ap = macro_ap_score(y_te, proba, list(classes))

    print(title)
    print("Macro-F1:", round(cr["macro avg"]["f1-score"], 4))
    print("Macro-Precision:", round(cr["macro avg"]["precision"], 4))
    print("Macro-Recall:", round(cr["macro avg"]["recall"], 4))
    print("Macro AP (OvR):", round(ap, 4))
    print("Confusion Matrix (rows=true, cols=pred):\n", cm)

    return {"classification_report": cr, "confusion_matrix": cm.tolist(), "macro_ap": ap}

base_metrics = holdout_report(calibrated, X_test, y_test, calibrated.classes_, title="[Baseline Calibrated]")

y_thr, proba_thr = predict_with_thresholds(calibrated, X_test, thresholds, calibrated.classes_)
cr_thr = classification_report(y_test, y_thr, output_dict=True)
cm_thr = confusion_matrix(y_test, y_thr, labels=calibrated.classes_)
ap_thr = macro_ap_score(y_test, proba_thr, list(calibrated.classes_))

print("\n[Thresholded for HIGH]")
print("Macro-F1:", round(cr_thr["macro avg"]["f1-score"], 4))
print("Macro-Precision:", round(cr_thr["macro avg"]["precision"], 4))
print("Macro-Recall:", round(cr_thr["macro avg"]["recall"], 4))
print("Macro AP (OvR):", round(ap_thr, 4))
print("Confusion Matrix:\n", cm_thr)


[Baseline Calibrated]
Macro-F1: 0.7087
Macro-Precision: 0.8818
Macro-Recall: 0.6679
Macro AP (OvR): 0.7923
Confusion Matrix (rows=true, cols=pred):
 [[ 11   0  27]
 [  0 305  40]
 [  0  81 396]]

[Thresholded for HIGH]
Macro-F1: 0.7325
Macro-Precision: 0.7828
Macro-Recall: 0.7069
Macro AP (OvR): 0.7923
Confusion Matrix:
 [[ 16   0  22]
 [  0 305  40]
 [  7  81 389]]


Cell 11 — Persist model bundle + reports (versioned, self-describing)

In [11]:
bundle = {
    "model": calibrated,
    "features": FEAT_COLS,
    "classes": list(calibrated.classes_),
    "thresholds": thresholds,
    "version": MODEL_VERSION,
    "config_used": str(cfg_path.name),
    "notes": "Calibrated (isotonic); thresholds tuned to emphasize HIGH recall."
}
path_model = MODEL_REG / "model1_risk_classifier.joblib"
joblib.dump(bundle, path_model)

# Save reports
with open(HOLD / "model1_holdout_baseline.json", "w") as f:
    json.dump(base_metrics, f, indent=2)

np.savetxt(HOLD / "model1_holdout_cm_baseline.csv",
           np.array(base_metrics["confusion_matrix"]), fmt="%d", delimiter=",")

with open(HOLD / "model1_thresholds.json", "w") as f:
    json.dump({"thresholds": thresholds}, f, indent=2)

print("Saved:")
print(" -", path_model)
print(" -", HOLD / "model1_holdout_baseline.json")
print(" -", HOLD / "model1_holdout_cm_baseline.csv")
print(" -", HOLD / "model1_thresholds.json")


Saved:
 - C:\Users\PC\Documents\Machine_Learning\Capstone_Project\Poseidon\artifacts\model_registry\model1_risk_classifier.joblib
 - C:\Users\PC\Documents\Machine_Learning\Capstone_Project\Poseidon\artifacts\holdout_reports\model1_holdout_baseline.json
 - C:\Users\PC\Documents\Machine_Learning\Capstone_Project\Poseidon\artifacts\holdout_reports\model1_holdout_cm_baseline.csv
 - C:\Users\PC\Documents\Machine_Learning\Capstone_Project\Poseidon\artifacts\holdout_reports\model1_thresholds.json


Cell 12 — Quick global importance (if supported) or note

In [12]:
def feature_importance(model_or_pipe, feat_names):
    # Try to locate a feature_importances_ attribute (Tree-based models)
    try:
        if hasattr(model_or_pipe, "feature_importances_"):
            return pd.Series(model_or_pipe.feature_importances_, index=feat_names).sort_values(ascending=False)
        # If wrapped by CalibratedClassifierCV, try the base_estimator
        if hasattr(model_or_pipe, "base_estimator"):
            est = model_or_pipe.base_estimator
            if hasattr(est, "feature_importances_"):
                return pd.Series(est.feature_importances_, index=feat_names).sort_values(ascending=False)
        if hasattr(model_or_pipe, "estimator"):
            est = model_or_pipe.estimator
            if hasattr(est, "feature_importances_"):
                return pd.Series(est.feature_importances_, index=feat_names).sort_values(ascending=False)
    except Exception:
        pass
    return None

imp = feature_importance(calibrated, FEAT_COLS)
if imp is not None:
    print("Global feature importances:")
    display(imp.to_frame("importance"))
    imp.to_csv(EXPL/"model1_feature_importance.csv")
else:
    print("Feature importances not available for this classifier. Use SHAP later (05 notebook).")


Global feature importances:


,importance
turbidity_proxy,0.261728
temperature,0.258812
predicted_nh3,0.167507
predicted_do,0.163909
pH,0.148043


Cell 13 — Local sensitivity (quick “why” proxy; SHAP later)

In [13]:
def local_sensitivity(model, x_row, feat_cols, delta=0.1):
    base_proba = model.predict_proba(x_row.values.reshape(1,-1))[0]
    rows = []
    idx = {c:i for i,c in enumerate(calibrated.classes_)}
    for c in feat_cols:
        x2 = x_row.copy()
        step = delta * (abs(x_row[c]) + 1e-6)  # small relative change
        x2[c] = x2[c] + step
        p2 = model.predict_proba(x2.values.reshape(1,-1))[0]
        rows.append({
            "feature": c, "delta": float(step),
            "dP_HIGH": float(p2[idx["HIGH"]] - base_proba[idx["HIGH"]]) if "HIGH" in idx else 0.0,
            "dP_MED" : float(p2[idx["MEDIUM"]] - base_proba[idx["MEDIUM"]]) if "MEDIUM" in idx else 0.0,
            "dP_LOW" : float(p2[idx["LOW"]] - base_proba[idx["LOW"]]) if "LOW" in idx else 0.0
        })
    return pd.DataFrame(rows).sort_values("dP_HIGH", ascending=False)

sample_idx = X_test.index[0]
sens_df = local_sensitivity(calibrated, X_test.loc[sample_idx], FEAT_COLS, delta=0.1)
print("Local sensitivity (increase each feature by ~10% absolute):")
display(sens_df)

Local sensitivity (increase each feature by ~10% absolute):


,feature,delta,dP_HIGH,dP_MED,dP_LOW
0,temperature,2.678594,0.0,0.000000,0.000000
1,pH,0.809201,0.0,0.531605,-0.531605
2,turbidity_proxy,0.001580,0.0,0.000000,0.000000
3,predicted_do,0.414490,0.0,-0.010760,0.010760
4,predicted_nh3,0.002467,0.0,-0.010760,0.010760
